# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [3]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# consultando as dimensões do df
print(sinasc.shape)

# Não há duplicados retornando o tamanho/dimensão original do df
sinasc.drop_duplicates().shape


(27028, 69)


(27028, 69)

In [4]:
# Confirmando dados da tabela não possuem linhas duplicadas
sinasc.index

RangeIndex(start=0, stop=27028, step=1)

In [5]:
# 2) Consultando dados do dataframe
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [6]:
# Contando a qtde de valores missing por variável
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [7]:
# Criação de uma base com a seleção das colunas de interesse
sinasc_resumo = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_resumo.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [14]:
sinasc_resumo.shape

(27028, 9)

In [15]:
# Fazendo a contagem de valores missing com as variáveis de interesse
sinasc_resumo.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [16]:
# 6) Validando a contagem da variável APGAR5 com valores missing
sinasc_resumo['APGAR5'].isna().sum()

103

In [17]:
sinasc['APGAR5'].isna().sum()

103

In [18]:
# Removendo todos os registros com Apgar5 não preenchido
sinasc_apgar5_preenchido = sinasc_resumo.dropna(subset=['APGAR5'])

In [19]:
# Validando a exclusão dos registros
sinasc_apgar5_preenchido.shape

(26925, 9)

In [20]:
# Validando a exclusão dos registros
sinasc_apgar5_preenchido.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [21]:
# Realizando consulta da qtde de registros com valor = 9
sinasc_count = (sinasc_apgar5_preenchido['ESTCIVMAE'] == 9).sum()
sinasc_count

144

In [22]:
# Realizando consulta da qtde de registros com valor = 9
sinasc_count = (sinasc_apgar5_preenchido['CONSULTAS'] == 9).sum()
sinasc_count

50

In [23]:
# Realizando consulta da qtde de registros com valor = 9
sinasc_apgar5_preenchido = sinasc_apgar5_preenchido[sinasc_apgar5_preenchido['ESTCIVMAE'] != 9]
sinasc_apgar5_preenchido.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0


In [24]:
sinasc_apgar5_preenchido.shape

(26781, 9)

In [25]:
# Realizando consulta da qtde de registros com valor != 9
sinasc_apgar5_preenchido = sinasc_apgar5_preenchido[sinasc_apgar5_preenchido['CONSULTAS'] != 9]
sinasc_apgar5_preenchido.shape

(26731, 9)

In [26]:
# Consultando isnull para ESTCIVMAE
sinasc_apgar5_preenchido.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         288
QTDFILVIVO    1554
GESTACAO      1197
GRAVIDEZ        75
CONSULTAS        0
APGAR5           0
dtype: int64

In [27]:
# Criar uma cópia do DataFrame
sinasc_apgar5_preenchido_copy = sinasc_apgar5_preenchido.copy()

In [28]:
# Removendo linhas isnull
sinasc_apgar5_preenchido_copy.dropna(subset=['ESTCIVMAE'], inplace=True)

In [29]:
# Confirmando a exclusão na variável ESTCIVMAE
sinasc_apgar5_preenchido_copy.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         242
QTDFILVIVO    1525
GESTACAO      1174
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64

In [30]:
sinasc_apgar5_preenchido_copy.shape

(26416, 9)

In [31]:
# Substituindo os valores faltantes (QTDFILVIVO) por zero.
sinasc_apgar5_preenchido_copy['QTDFILVIVO'].fillna(0, inplace=True)

In [32]:
# Validando execução do script acima
sinasc_apgar5_preenchido_copy.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         242
QTDFILVIVO       0
GESTACAO      1174
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64

In [33]:
# Realizando análise na variável ESCMAE
sinasc_apgar5_preenchido_copy['ESCMAE'].value_counts()

ESCMAE
8 a 11 anos        15332
12 anos ou mais     5654
4 a 7 anos          4645
1 a 3 anos           504
Nenhuma               39
Name: count, dtype: int64

In [34]:
# Realizando análise na variável ESCMAE
sinasc_apgar5_preenchido_copy['ESCMAE'].count()

26174

In [35]:
# Realizando análise na variável ESCMAE
sinasc_apgar5_preenchido_copy[sinasc_apgar5_preenchido_copy['ESCMAE'].isnull()]

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
38,1,25,1.0,NaN,3.0,37 a 41 semanas,Única,4,8.0
66,1,23,5.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
93,1,22,5.0,NaN,2.0,37 a 41 semanas,Única,3,9.0
229,1,25,1.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
447,1,21,5.0,NaN,0.0,37 a 41 semanas,Única,3,10.0
...,...,...,...,...,...,...,...,...,...
26604,1,23,2.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
26605,1,18,5.0,NaN,1.0,37 a 41 semanas,Única,4,10.0
26765,1,30,2.0,NaN,2.0,37 a 41 semanas,Única,4,8.0
26902,1,22,5.0,NaN,1.0,37 a 41 semanas,Única,3,10.0


In [36]:
# Realizando análise na variável ESCMAE
242/26174

0.009245816459081532

In [37]:
# Após análise, decisão tomanda em excluir linhas isnull da variável ESCMAE
sinasc_apgar5_preenchido_copy.dropna(subset=['ESCMAE'], inplace=True)

In [38]:
sinasc_apgar5_preenchido_copy.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1150
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64

In [39]:
# Após análise, decisão em excluir linhas isnull da variável GRAVIDEZ
sinasc_apgar5_preenchido_copy.dropna(subset=['GRAVIDEZ'], inplace=True)

In [40]:
sinasc_apgar5_preenchido_copy.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE           0
QTDFILVIVO       0
GESTACAO      1134
GRAVIDEZ         0
CONSULTAS        0
APGAR5           0
dtype: int64

In [41]:
# Após análise, e por causa da qtde de registros is null da variável GESTACAO, foram mantidas as linhas
# mas substituindo is null para 0
sinasc_apgar5_preenchido_copy['GESTACAO'].fillna(0, inplace=True)

In [42]:
sinasc_apgar5_preenchido_copy.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [43]:
sinasc_apgar5_preenchido_copy.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0


In [44]:
sinasc_apgar5_preenchido_copy.shape

(26101, 9)

In [45]:
sinasc_apgar5_preenchido_copy['APGAR5'].value_counts()

APGAR5
9.0     15592
10.0     8144
8.0      1919
7.0       233
6.0        76
5.0        44
1.0        24
4.0        21
2.0        17
0.0        16
3.0        15
Name: count, dtype: int64

In [46]:
# Criando uma nova variável de acordo com a categoriazação da variável APGAR5
sinasc_apgar5_preenchido_copy.loc[sinasc_apgar5_preenchido_copy['APGAR5'] <= 3, 'APGAR5_FAIXA'] = 'asfixia severa'
sinasc_apgar5_preenchido_copy.loc[(sinasc_apgar5_preenchido_copy['APGAR5'] >= 4) & (sinasc_apgar5_preenchido_copy['APGAR5'] < 6),
           'APGAR5_FAIXA'] = 'asfixia moderada'
sinasc_apgar5_preenchido_copy.loc[(sinasc_apgar5_preenchido_copy['APGAR5'] >= 6) & (sinasc_apgar5_preenchido_copy['APGAR5'] <= 7),
           'APGAR5_FAIXA'] = 'asfixia leve'
sinasc_apgar5_preenchido_copy.loc[(sinasc_apgar5_preenchido_copy['APGAR5'] > 7), 'APGAR5_FAIXA'] = 'normal'

In [47]:
# Consulta Dataframe
sinasc_apgar5_preenchido_copy.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_FAIXA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal


In [49]:
# Consulta de valores na variável APGAR5
sinasc_apgar5_preenchido_copy['APGAR5'].value_counts()

APGAR5
9.0     15592
10.0     8144
8.0      1919
7.0       233
6.0        76
5.0        44
1.0        24
4.0        21
2.0        17
0.0        16
3.0        15
Name: count, dtype: int64

In [50]:
# Consultando frequência na nova variável APGAR5_FAIXA
sinasc_apgar5_preenchido_copy['APGAR5_FAIXA'].value_counts()

APGAR5_FAIXA
normal              25655
asfixia leve          309
asfixia severa         72
asfixia moderada       65
Name: count, dtype: int64

In [51]:
# Consultando frequência em porcentagem na nova variável APGAR5_FAIXA
sinasc_apgar5_preenchido_copy['APGAR5_FAIXA'].value_counts() / 26101

APGAR5_FAIXA
normal              0.982913
asfixia leve        0.011839
asfixia severa      0.002759
asfixia moderada    0.002490
Name: count, dtype: float64

In [52]:
# Renomeando as variáveis para o formato snake case
columns_rename = []
for column in sinasc_apgar5_preenchido_copy:
    columns_rename.append(column.lower().replace(' ', '-'))
sinasc_apgar5_preenchido_copy.columns = columns_rename
sinasc_apgar5_preenchido_copy.head()

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar5_faixa
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
